This notebook takes as input a flat event log stored in a CSV file and groupe the attribute values (i.e., orders objects)  as a multiset if the activity name, timestamp and weight, price attribute have the same value. As output a new CSV file is created called 'orders_case_notion_event_log_grouped'. The following query can be used if we want to apply the same idea using query statement. 

```
SELECT activity_name, timestamp, weight, price,
ARRAY_AGG(DISTINCT numbers) AS orders object
FROM orders_case_notion_event_log
GROUP BY activity_name, timestamp, weight, price;
```

In [1]:
%%capture
!pip install pandas
!pip install pm4py

In [2]:
import pandas as pd
import csv
import pm4py

In [3]:
df_orders = pd.read_csv('./input/orders_case_notion_event_log.csv')

In [4]:
# select the specified columns and order the results for the 
selected_cols = ["number", "ocel activity", "ocel timestamp", "weight_y", "price_y"]
ordered_results = df_orders[selected_cols].sort_values(["ocel timestamp", "weight_y", "price_y"])

# print the results
print(ordered_results)

           number      ocel activity       ocel timestamp  weight_y  price_y
0      ['990001']        place order  2019-05-20 09:07:47     3.520   524.96
1      ['990002']        place order  2019-05-20 10:35:21     2.656  3255.99
2      ['990002']          pick item  2019-05-20 10:38:17     0.483    79.99
3      ['990001']      confirm order  2019-05-20 11:13:54     3.520   524.96
4      ['990001']          pick item  2019-05-20 11:20:13     0.280    89.99
...           ...                ...                  ...       ...      ...
32442  ['991976']       send package  2020-08-18 11:11:09     0.606  1275.00
32443  ['991976']  package delivered  2020-08-19 17:57:32     0.606  1275.00
32444  ['991983']     create package  2020-08-22 01:00:00     0.172   699.00
32445  ['991983']       send package  2020-08-24 11:14:47     0.172   699.00
32446  ['991983']  package delivered  2020-08-25 16:30:41     0.172   699.00

[32447 rows x 5 columns]


In [5]:
# group the rows with the same values in the specified columns
groups = df_orders.groupby(["ocel activity", "ocel timestamp", "weight_y", "price_y"])

# initialize the group number and the row number to 1
group_number = 1
row_number = 1

# create a new column "row_number" and mark the rows in each group with a sequential number
df_orders["row_number"] = 0

for group_name, group_df in groups:
    # mark the rows in the current group with a sequential row number
    df_orders.loc[group_df.index, "row_number"] = row_number
    # increment the row number for the next row
    row_number += 1
    # update the group number if the current group ends
    if len(group_df) == 1:
        group_number += 1
    # mark the rows in the current group with the current group number
    df_orders.loc[group_df.index, "group_number"] = group_number

# print the updated data frame with the row and group numbers
print(df_orders)


       source_id      number       ocel timestamp      ocel activity  weight   
0            1.0  ['990001']  2019-05-20 09:07:47        place order     NaN  \
1            2.0  ['990002']  2019-05-20 10:35:21        place order     NaN   
2            3.0  ['990002']  2019-05-20 10:38:17          pick item     NaN   
3            4.0  ['990001']  2019-05-20 11:13:54      confirm order     NaN   
4            5.0  ['990001']  2019-05-20 11:20:13          pick item     NaN   
...          ...         ...                  ...                ...     ...   
32442    22363.0  ['991976']  2020-08-18 11:11:09       send package     NaN   
32443    22364.0  ['991976']  2020-08-19 17:57:32  package delivered     NaN   
32444    22365.0  ['991983']  2020-08-22 01:00:00     create package     NaN   
32445    22366.0  ['991983']  2020-08-24 11:14:47       send package     NaN   
32446    22367.0  ['991983']  2020-08-25 16:30:41  package delivered     NaN   

       price  weight_y  price_y  row_nu

In [6]:
# save the updated data frame to a new CSV file
df_orders.to_csv("orders_case_notion_event_log_with_numbers.csv", index=False)

In [7]:
# group the data frame by source_id, ocel timestamp, acel activity, weight_y, and price_y
groups = df_orders.groupby(["source_id", "ocel timestamp", "ocel activity", "weight_y", "price_y"])["number"].agg(list)

# create a new data frame with the group information and the concatenated numbers
df_grouped = groups.reset_index().groupby("source_id").agg({
    "source_id":list,
    "ocel timestamp": list,
    "ocel activity": list,
    "weight_y": list,
    "price_y": list,
    "number": lambda x: [num for group in x for num in group]
})

# rename the columns in the new data frame
df_grouped.columns = ["source_id","ocel timestamp", "ocel activity", "weight_y", "price_y", "numbers"]

# Modify the data structures before saving
df_grouped["numbers"] = df_grouped["numbers"].apply(lambda lst: ','.join(map(str, lst)))
df_grouped["source_id"] = df_grouped["source_id"].apply(lambda lst: ','.join(map(str, lst)))
df_grouped["ocel timestamp"] = df_grouped["ocel timestamp"].apply(lambda lst: ','.join(map(str, lst)))
df_grouped["ocel activity"] = df_grouped["ocel activity"].apply(lambda lst: ','.join(map(str, lst)))
df_grouped["weight_y"] = df_grouped["weight_y"].apply(lambda lst: ','.join(map(str, lst)))
df_grouped["price_y"] = df_grouped["price_y"].apply(lambda lst: ','.join(map(str, lst)))

# Save the updated data frame to a new CSV file
df_grouped.to_csv("orders_case_notion_event_log_grouped.csv", index=False)


# print a message to confirm that the file was saved
print("Data frame saved to orders_case_notion_event_log_grouped.csv")

Data frame saved to orders_case_notion_event_log_grouped.csv
